In [6]:
using Plots

In [7]:
include("atlas_option.jl")

[2.236435731611934, 1.8260366016126244, 2.6468348616112434]

In [8]:
function compare_variance(num_of_sim::Int, α::Float64, T::Int, N::Int, r::Float64, K::Float64, basket_volume::Int, 
                        S₀::Array{Float64}, sigma::Array{Float64}, 
                        correlation_matrix::Matrix{Float64}, n1::Int, n2::Int)


    normal = atlas_option_monte_carlo(num_of_sim, α, T, N, r, K, basket_volume, S₀, sigma, correlation_matrix, 
                                    n1, n2, "basic")
    antithetic = atlas_option_monte_carlo(num_of_sim, α, T, N, r, K, basket_volume, S₀, sigma, 
                                        correlation_matrix, n1, n2,"antithetic")
    quasi = atlas_option_monte_carlo(num_of_sim, α, T, N, r, K, basket_volume, S₀, sigma, correlation_matrix, 
                                    n1, n2, "quasi_monte_carlo")
    moment_matching = price_atlas_moment_matching(num_of_sim, α, T, N, r, K, basket_volume, S₀, sigma, 
                                    correlation_matrix, n1, n2) 
    LHS = atlas_option_monte_carlo(num_of_sim, α, T, N, r, K, basket_volume, S₀, sigma, correlation_matrix, 
                                    n1, n2, "LHS")

    mean_elements = normal[1]
    bottom_elements = normal[2]
    upper_elements = normal[3]

    mean_antithetic = antithetic[1]
    bottom_antithetic = antithetic[2]
    upper_antithetic = antithetic[3]

    mean_quasi = quasi

    mean_moment_matching = moment_matching[1]
    bottom_moment_matching = moment_matching[2]
    upper_moment_matching = moment_matching[3]

    mean_LHS = LHS[1]
    bottom_LHS = LHS[2]
    upper_LHS = LHS[3]

    length_normal = upper_elements-bottom_elements
    length_antithetic = upper_antithetic - bottom_antithetic
    length_moment_matching = upper_moment_matching - bottom_moment_matching
    length_LHS = upper_LHS-bottom_LHS

    return [mean_elements, mean_antithetic, mean_moment_matching, mean_LHS, mean_quasi, 
            length_normal, length_antithetic, length_moment_matching, length_LHS]
end

function num_of_sims(num_of_sims::Array{Int},α::Float64, T::Int, N::Int, r::Float64, K::Float64, 
                    basket_volume::Int, S₀::Array{Float64}, sigma::Array{Float64}, 
                    correlation_matrix::Matrix{Float64}, n1::Int, n2::Int)

    results::Matrix{Float64} = zeros(length(num_of_sims), 9)
    for (ind, num_of_sim) in enumerate(num_of_sims)
        results[ind,:] = compare_variance(num_of_sim,α, T, N, r, K, basket_volume, S₀, sigma, correlation_matrix, 
                                        n1, n2)
    end

    return results

end 


function interest_rate(num_of_sims::Int,α::Float64, T::Int, N::Int, rs::Array{Float64}, K::Float64, 
                        basket_volume::Int, S₀::Array{Float64},  sigma::Array{Float64}, 
                        correlation_matrix::Matrix{Float64}, n1::Int, n2::Int)


    results::Matrix{Float64} = zeros(length(rs), 9)
    for (ind, r) in enumerate(rs)
        results[ind,:] = compare_variance(num_of_sims,α, T, N, r, K, basket_volume, S₀, sigma, correlation_matrix, 
        n1, n2)
    end

    return results

end 


function maturity(num_of_sims::Int,α::Float64, T::Int, r::Float64, K::Float64, basket_volume::Int, S₀::Array{Float64}, 
                 sigma::Array{Float64}, correlation_matrix::Matrix{Float64}, n1::Int, n2::Int)


    results::Matrix{Float64} = zeros(T, 9)
    for maturity in 1:T
        results[maturity,:] = compare_variance(num_of_sims,α, maturity, maturity*250, r, K, basket_volume, S₀, 
                                                sigma, correlation_matrix, n1, n2)
    end

    return results

end


function correlation(num_of_sims::Int,α::Float64, T::Int, N::Int, r::Float64, K::Float64, basket_volume::Int, 
                    S₀::Array{Float64}, sigma::Array{Float64}, n1::Int, n2::Int)

    corr_matrices::Vector{Matrix{Float64}} = [[1.0 0.1 0.1; 0.1 1.0 0.1; 0.1 0.1 1.0], 
                                              [1.0 0.5 0.4; 0.5 1.0 0.6; 0.4 0.6 1.0], 
                                              [1.0 0.9 0.8; 0.9 1.0 0.7; 0.8 0.7 1.0]] 


    results::Matrix{Float64} = zeros(3, 9)
    for correlation in eachindex(corr_matrices)
        results[correlation,:] = compare_variance(num_of_sims,α, T, N, r, K, basket_volume, S₀, sigma, 
                                                    corr_matrices[correlation], n1, n2)
    end
    return results

end


function plot_variance(vals, variance, title, xlabel)
    scatter(vals,variance, label = ["normal" "antithetic" "moment_matching" "LHS"], title = title, legend = :outertop)
    xlabel!(xlabel)
    savefig(string("C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\atlas\\", 
                    title, "_atlas_", ".svg"))
end

function plot_mean(vals, mean, title, xlabel)
    scatter(vals, mean, label=["mean_elements" "mean_antithetic" "mean_moment_matching" "mean_LHS" "mean_quasi"], 
            legend = :outertop)
    title!(title)
    xlabel!(xlabel)
    savefig(string("C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\atlas\\", 
            title,"_atlas_", ".svg"))
end

function plot_bar_plot(results,name)
    bar(["słaba", "średnia", "silna"],results,label=name,
        title=string("długość przedziału ~ korelacja dla metody ",name), 
        xlabel = "siła korelacji między spółkami", 
        xticks = :all,legend=false, fmt = :svg)    
    savefig(string("C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\atlas\\", name, 
            "_atlas_", ".svg"))
end

plot_bar_plot (generic function with 1 method)

In [ ]:
function compare_pricing_time(num_of_sim::Int, α::Float64, T::Int, N::Int, r::Float64, K::Float64, basket_volume::Int, 
                                S₀::Array{Float64}, sigma::Array{Float64}, 
                                correlation_matrix::Matrix{Float64}, n1::Int, n2::Int)


t_normal = @elapsed [atlas_option_monte_carlo(num_of_sim, α, T, N, r, K, basket_volume, S₀, sigma, correlation_matrix, n1, n2, "basic") for i in 1:1000]
t_antithetic = @elapsed  [atlas_option_monte_carlo(num_of_sim, α, T, N, r, K, basket_volume, S₀, sigma, correlation_matrix, n1, n2,,"antithetic") for i in 1:1000]
t_quasi_monte_carlo = @elapsed [atlas_option_monte_carlo(num_of_sim, α, T, N, r, K, basket_volume, S₀, sigma, correlation_matrix, n1, n2, "quasi_monte_carlo") for i in 1:1000]
t_moment_matching = @elapsed [price_atlas_moment_matching(num_of_sim, α, T, N, r, K, basket_volume, S₀, sigma, correlation_matrix, n1, n2,) for i in 1:1000] 
t_LHS = @elapsed [atlas_option_monte_carlo(num_of_sim, α, T, N, r, K, basket_volume, S₀, sigma, correlation_matrix, n1, n2,"LHS") for i in 1:1000]

return Dict("basic" => t_normal, "antithetic" => t_antithetic, "quasi_monte_carlo" => t_quasi_monte_carlo,
"moment_matching" => t_moment_matching, "LHS" => t_LHS)
end


In [9]:
cov_matrix = [1.0 0.3 0.4; 0.3 1.0 0.1; 0.4 0.1 1.0] 

3×3 Matrix{Float64}:
 1.0  0.3  0.4
 0.3  1.0  0.1
 0.4  0.1  1.0

In [17]:
results = compare_variance(2000,0.01,2,500,0.03,8.0,3,[10.0,10.0,10.0], [0.15,0.15,0.15],cov_matrix,1,1)[1:5]

5-element Vector{Float64}:
 2.3783578420006877
 2.3764371798790025
 2.4943041214015182
 2.2433961032593737
 2.436923060426837

In [18]:
(results[2:5] .- results[1])./results[1] # mean_antithetic, mean_moment_matching, mean_LHS, mean_quasi

4-element Vector{Float64}:
 -0.0008075580922967874
  0.04875056114486789
 -0.05674576649398707
  0.024624224913473904

# Ilość symulacji

In [16]:
n_of_sims = [100:100:5000;]
sims::Matrix{Float64} = num_of_sims(n_of_sims,0.01,2,500,0.03,8.0,3,[10.0,10.0,10.0],
                                    [0.15,0.15,0.15],cov_matrix,1,1)
variance = sims[:,6:end]
plot_variance(n_of_sims,variance,"długość przedziału ufności ~ wielkość próby", "wielkość próby")
means = sims[:,1:5]
plot_mean(n_of_sims,means,"cena opcji ~ wielkość próby", "wielkość próby")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\atlas\\cena opcji ~ wielkość próby_atlas_.svg"

# Stopy procentowe

In [13]:
rs = [0.01:0.001:0.1;]
sims_rs::Matrix{Float64} = interest_rate(1000,0.01,2,500,rs, 8.0,3,[10.0,10.0,10.0],
                                        [0.15,0.15,0.15], cov_matrix, 1, 1)
variance_rs = sims_rs[:,6:end]
plot_variance(rs,variance_rs, "długość przedziału ufności ~ stopy procentowe", "stopy procentowe")
means_rs = sims_rs[:,1:5]
plot_mean(rs,means_rs, "cena opcji ~ stopy procentowe", "stopy procentowe")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\atlas\\cena opcji ~ stopy procentowe_atlas_.svg"

# Matura

In [14]:
sims_maturity::Matrix{Float64} = maturity(1000,0.01,10,0.03,8.0,3,[10.0,10.0,10.0],
                                        [0.15,0.15,0.15],cov_matrix, 1, 1)
mat= [i for i in 1:10]
variance_maturity = sims_maturity[:,6:end]
plot_variance(mat,variance_maturity, "długość przedziału ~ matura", "matura")
means_maturity = sims_maturity[:,1:5]
plot_mean(mat,means_maturity, "cena opcji ~ matura", "matura")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\atlas\\cena opcji ~ matura_atlas_.svg"

# Korelacja

In [15]:
sims_correlation::Matrix{Float64} = correlation(1000,0.01,2,500,0.03,8.0,3,[10.0,10.0,10.0],
                                                [0.15,0.15,0.15],1,1)
plot_bar_plot(sims_correlation[:,6],"basic")
plot_bar_plot(sims_correlation[:,7],"antithetic")
plot_bar_plot(sims_correlation[:,8],"moment_matching")
plot_bar_plot(sims_correlation[:,9],"LHS")

"C:\\Users\\adama\\OneDrive\\Pulpit\\magisterka\\optionPricing\\graphs\\atlas\\LHS_atlas_.svg"